In [1]:
from google.colab import files
upl = files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [2]:
import altair as alt
import pandas as pd

# loads data
df_cleaned = pd.read_csv("cleaned_rental_data.csv")

# identifies amenity columns
amenity_cols = [col for col in df_cleaned.columns if col not in [
    "Rental Type", "Bedrooms", "Bathrooms", "Furnished", "Neighborhood",
    "Proximity to Transit", "Total Rent Avg", "Utility Cost Avg",
    "Personal Income Avg", "People in Unit", "Total Rent w/ Utilities"
]]

# melts to long format for grouping and filtering
df_long = df_cleaned.melt(
    id_vars=["Total Rent w/ Utilities"],
    value_vars=amenity_cols,
    var_name="Amenity",
    value_name="Has Amenity"
)

# only keeps rows where the amenity is present
df_long = df_long[df_long["Has Amenity"] == 1]

# x axis encoding
x_axis = alt.X('Amenity:N', sort='-y', title='Amenity', axis=alt.Axis(labelAngle=-45))

# creates boxplot layer
boxplot = alt.Chart(df_long).mark_boxplot(color="#0b3d91").encode(
    x=x_axis,
    y=alt.Y('Total Rent w/ Utilities:Q', title='Total Rent')
)

# creates transparent bars with rent stats based on amenity
tooltips = alt.Chart(df_long).transform_aggregate(
    count='count()',
    mean='mean(Total Rent w/ Utilities)',
    q1='q1(Total Rent w/ Utilities)',
    median='median(Total Rent w/ Utilities)',
    q3='q3(Total Rent w/ Utilities)',
    groupby=['Amenity']
).mark_bar(opacity=0).encode(
    x=x_axis,
    y=alt.Y('median:Q'),
    tooltip=[
        alt.Tooltip('Amenity:N', title='Amenity'),
        alt.Tooltip('count:Q', title='Number of Listings'),
        alt.Tooltip('mean:Q', title='Mean Rent', format=',.0f'),
        alt.Tooltip('q1:Q', title='1st Quartile', format=',.0f'),
        alt.Tooltip('median:Q', title='Median Rent', format=',.0f'),
        alt.Tooltip('q3:Q', title='3rd Quartile', format=',.0f')
    ]
)

# combines chart layers for final chart
final = (boxplot + tooltips).properties(
    width=600,
    height=400,
    title="Rent Distribution by Amenity"
)

final

alt.LayerChart(...)

In [3]:
# Converts the chart to html format and creates the file
html = final.to_html(embed_options={'actions': False})

with open('amenity_rent.html', 'w') as f:
    f.write(html)

In [ ]:
from google.colab import files
files.download('amenity_rent.html')